In [332]:

import image_reader as ir
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import random

def plot_pdf(intensities, domain):
	plt.figure(figsize=(8, 8))
	plt.plot(np.linspace(0, domain, len(intensities)), intensities, color='magenta', linewidth=2)
	plt.xlabel(r'$\theta$', fontsize=22)
	plt.ylabel('Energy p(X)', fontsize=22)
	plt.title('Probability Density Function (PDF) of Rows', fontsize=22, pad=25)
	plt.grid(True, color='white', alpha=.5)
	plt.gca().set_facecolor('black')
	plt.tick_params(axis='both', which='major', labelsize=14)
	plt.xlim(0, domain)
	plt.ylim(0, np.max(intensities))
	# plt.xticks(np.arange(0, 1.1, 0.1))
	y_increment = np.max(intensities) / 10
	plt.yticks(np.arange(0, np.max(intensities) + y_increment, y_increment))
	plt.show()

def plot_cdf(cdf, domain):
	plt.figure(figsize=(8, 8))
	plt.plot(np.linspace(0, domain, len(cdf)), cdf, color='magenta', linewidth=2)
	plt.xlabel(r'$\theta$', fontsize=22)
	plt.ylabel('Energy p(X)', fontsize=22)
	plt.title('Cumulative Distribution Function (CDF) of Intensities', fontsize=22, pad=25)
	plt.grid(True, color='white', alpha=.5)
	plt.gca().set_facecolor('black')
	plt.tick_params(axis='both', which='major', labelsize=14)
	plt.xlim(0, domain)
	plt.ylim(0, 1)
	plt.yticks(np.arange(0, 1.1, 0.1))
	plt.show()

def gamma_correction(image, gamma=2.2):
	return np.power(image, 1 / gamma)

def pfm_to_ppm(image, scaling_factor=1):

    return (gamma_correction(image / scaling_factor, 1.2) * 255).astype(np.uint16)

In [268]:
grace_path = os.path.join("..", "GraceCathedral", "grace_latlong.pfm")
grace_pfm = ir.read_pfm(grace_path)
print(grace_pfm.shape)
pixel_extent = 9


(512, 1024, 3)


In [333]:
intensities = [np.sum(grace_pfm[i]) * math.sin(math.pi * i / grace_pfm.shape[0]) for i in range(grace_pfm.shape[0])]
intensities /= np.sum(intensities)
cdf = np.cumsum(intensities)
# plot_pdf(intensities, math.pi)
# plot_cdf(cdf, math.pi)

scaling_factor = np.max(grace_pfm)
print(scaling_factor)
for samples in [64,256,1024]:
	grace_sampled = grace_pfm.copy()
	if samples == 256:
		grace_copy = np.zeros(grace_pfm.shape).astype(np.float32)

	for i in range(samples):
		
		randrow = np.searchsorted(cdf, random.uniform(0, 1))

		sampled_row = [sum(p) for p in grace_pfm[randrow]]
		sampled_row /= np.sum(sampled_row)
		row_cdf = np.cumsum(sampled_row)

		randcol = np.searchsorted(row_cdf, random.uniform(0, 1))

		# plot_pdf(sampled_row, 2*math.pi)
		# plot_cdf(row_cdf, 2*math.pi)
		

		for i in range(pixel_extent):
			for j in range(pixel_extent):
				edge = pixel_extent // 2
				if randrow - edge + i >= 0 and randrow - edge + i < grace_pfm.shape[0] and randcol - edge + j >= 0 and randcol - edge + j < grace_pfm.shape[1]:
					grace_sampled[randrow - edge + i, randcol - edge + j] = [0, 0, 1000]
					if samples == 256:
						grace_copy[randrow - edge + i, randcol - edge + j] = grace_pfm[randrow - edge + i, randcol - edge + j]
	
	
	ir.write_pfm(gamma_correction(grace_sampled), f"output/grace_sampled_{samples}.pfm")
	ir.write_ppm(pfm_to_ppm(grace_sampled, scaling_factor), f"output/grace_sampled_{samples}.ppm")
 
	if samples == 256:
		ir.write_pfm(gamma_correction(grace_copy), f"output/grace_copy.pfm")
		ir.write_ppm(pfm_to_ppm(grace_copy, scaling_factor), f"output/grace_copy.ppm")




488.25
